# 7장. 딥러닝을 위한 고급 도구(Advacned Tools for Deep Learning)
※ 이 장에서 다룰 핵심 내용
* 케라스의 함수형 API
* 케라스 콜백 사용 방법
* 시각화 도구인 텐서보드 사용 방법
* 최고 수준의 모델을 만들기 위한 모범 사례

## 7.1 Sequential 모델을 넘어서: 케라스의 함수형 API
* 지금까지 소개한 모든 신경망은 Sequential 모델을 사용하여 만들었습니다. Sequential 모델은 네트워크 입력과 출력이 하나라고 가정합니다. 이 모델은 층을 차례대로 쌓아 구성합니다.

* 많은 경우에 이런 가정이 적절합니다. 이 책에서는 지금까지 Sequential 모델 클래스만 사용하여 많은 개념과 실제 application을 다루었습니다. 하지만 이런 가정이 맞지 않는 경우도 많습니다. 일부 네트워크는 개별 입력이 여러 개 필요하거나 출력이 여러 개 필요합니다. 층을 차례대로 쌓지 않고 층 사이를 연결하여 그래프처럼 만드는 네트워크도 있습니다.

* 예를 들어 어떤 작업은 다양한 종류의 입력이 필요합니다. 다양한 입력 소스(source)에서 전달된 데이터를 다른 종류의 신경망 층을 사용하여 처리하고 합칩니다. 중고 의류의 시장 가격을 예측하는 딥러닝 모델을 상상해 보죠. 이 모델은 사용자가 제공한 메타데이터(의류 브랜드, 연도 등), 사용자가 제공한 텍스트 설명, 제품 사진을 입력으로 사용합니다. 메타데이터만 있으면 이를 원-핫 인코딩으로 바꾸고 완전 연결 네트워크를 사용하여 가격을 예측할 수 있습니다. 텍스트 설명만 있다면 RNN이나 1D 컨브넷을 사용할 수 있습니다. 사진 이미지만 있다면 2D 컨브넷을 사용할 수 있습니다. 이 세 모델을 동시에 모두 사용할 수 있을까요? 간단한 방법은 3개의 모델을 따로 훈련하고 각 예측을 가중 평균(weighted average)하는 것입니다. 각 모델에서 추출한 정보가 중복된다면 이 방식은 최적이 아닐 것입니다. 가능한 모든 종류의 입력 데이터를 동시에 사용해서 정확한 하나의 모델을 학습하는 것이 더 나은 방법입니다. 이 모델은 3개의 입력 가지가 필요합니다.

* 이와 비슷하게 어떤 작업은 입력 데이터에서 여러 개의 타깃 속성을 예측해야 합니다. 예를 들어 소설이나 짧은 글이 있을 때 자동으로 장르별로 분류하려고 합니다(로맨스나 스릴러 등). 또 글을 쓴 대략의 시대를 예측해야 합니다. 물론 2개의 모델을 따로 훈련할 수 있습니다. 장르를 위한 모델과 시대를 위한 모델입니다. 하지만 이 속성들은 통계적으로 독립적이지 않기 때문에 동시에 장르와 시대를 함께 예측하도록 학습해야 더 좋은 모델을 만들 수 있습니다. 이 모델은 2개의 출력 또는 머리(head)를 가집니다. 장르와 시대 사이의 상관관계 때문에 소설 시대를 알면 장르의 공간에서 정확하고 풍부한 표현을 학습하는 데 도움이 됩니다. 그 반대도 마찬가지입니다.

* 더불어 최근에 개발된 많은 신경망 구조는 선형적이지 않은 네트워크 토폴로지(topology)가 필요합니다. 비순환 유향 그래프 같은 네트워크 구조입니다. 예를 들어 (구글의 세게대 등이 개발한)인셉션 모듈을 사용하는 인셉션 계열의 네트워크들입니다. 이 모듈에서 입력은 나란히 놓인 여러 개의 합성곱 층을 거쳐 하나의 텐서로 출력이 합쳐집니다.

* 최근에는 모델에 **잔차 연결**을 추가하는 경향도 있습니다. (마이크로소프트의 허(He) 등이 개발한) ResNet 계열의 네트워크들이 이런 방식을 사용하기 시작했습니다. 잔차 연결은 하위 층의 출력 텐서를 상위 층의 출력 텐서에 더해서 아래층의 표현이 네트워크 위쪽으로 흘러갈 수 있도록 합니다. 하위 층에서 학습된 정보가 데이터 처리 과정에서 손실되는 것을 방지합니다. 이렇게 그래프 구조를 띤 네트워크 종류가 많습니다.
★ 잔차 연결: 하위 층의 출력을 상위 층의 특성 맵에 더한다.
* 여러 경우에 다중 입력 모델, 다중 출력 모델, 그래프 구조를 띤 모델이 필요하지만 케라스의 Sequential 클래스를 사용해서는 만들지 못합니다. 케라스에는 훨씬 더 일반적이고 유연한 다른 방법인 **함수형 API**가 있습니다. 이 절에서 함수형 API가 무엇인지 소개하고, 함수형 API를 사용하는 방법과 이를 사용하여 할 수 있는 것을 자세히 설명하겠습니다.

### 7.1.1 함수형 API 소개
* 함수형 API(functional API)에서는 직접 텐서들의 입출력을 다룹니다. 함수처럼 층을 사용하여 텐서를 입력받고 출력합니다(그래서 함수형 API라고 부릅니다).

In [ ]:
from keras import Input, layers

# 텐서(tensor)
input_tensor = Input(shape=(32,))
# 함수처럼 사용하기 위해 층 객체를 만듭니다.
dense = layers. Dense(32, activation='relu')
# 텐서와 함께 층을 호출하면 텐서를 반환합니다.
output_tensor = dense(input_tensor)

* 간단한 예를 통해 Sequential 모델과 함수형 API로 만든 동일한 모델을 나란히 비교해 보겠습니다.

In [3]:
# Sequential 모델과 함수형 API로 만든 동일한 모델을 비교
# Compare Sequential model with the same model which created by functional API
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers
from tensorflow.keras import Input

seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

# 함수형 API로 만든 모델입니다.
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

# 입력과 출력 텐서를 지정하여 Model 클래스의 객체를 만듭니다.
model = Model(input_tensor, output_tensor)
# 모델 구조를 확인해 보죠!
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_5 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


* 입력 텐서와 출력 텐서만 가지고 Model 객체를 만드는 부분이 조금 마술처럼 보입니다. 무대 뒤에서 케라스는 input_tensor에서 output_tensor로 가는 데 필요한 모든 층을 추출합니다. 그다음 이들을 모아 그래프 데이터 구조인 Model 객체를 만듭니다. 물론 input_tensor를 반복 변환하여 output_tensor를 만들 수 있어야 됩니다. 관련되지 않은 입력과 출력으로 모델을 만들면 RuntimeError가 발생합니다.

```python
>>> unrelated_input = Input(shape=(32,))
>>> bad_model = model = Model(unrelated_input, output_tensor)
RuntimeError: Graph disconnected: cannot
obtain value for tensor
Tensor("input_1:0", shape=(?, 64), dtype=float32) at layer "input_1".
```
* 이 에러는 케라스가 출력 텐서에서 input_1 텐서로 다다를 수 없다는 뜻입니다.

* Model 객체를 사용한 컴파일, 훈련, 평가 API는 Sequential 클래스와 같습니다. 

In [ ]:
# Model 객체를 사용한 컴파일, 훈련, 평가 API 구현
# implement compile, training, evaluation API by using Model object 
# 모델을 컴파일합니다.
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# 훈련을 위해 랜덤한 넘파이 데이터를 생성합니다.
import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 64))
# 열 번 에포트 동안 모델을 훈련합니다.
model.fit(x_train, y_train, epochs=10, batch_size=128)
# 모델을 평가합니다.
score = model.evaluate(x_train, y_train)

### 7.1.2 다중 입력 모델(multi-input model)
* 함수형 API는 다중 입력 모델을 만드는 데 사용할 수 있습니다. 일반적으로 이런 모델은 서로 다른 입력 가지를 합치기 위해 여러 텐서를 연결할 수 있는 층을 사용합니다. 텐서를 더하거나 이어 붙이는 식입니다. 이와 관련된 케라스의 함수는 keras, layers.add, keras.layers.concatenate 등입니다. 아주 간단한 다중 입력 모델을 살펴보겠습니다. 질문-응답(question-answering) 모델입니다.

※ 이외에도 layers.average(), layers.maximum(), layers.minimum(), layers.multiply(), layers.subtract(), layers.dot()이 있습니다. 이 함수들은 tensorflow.keras.layers.merge 모듈 아래 정의되어 있습니다.

* 전형적인 질문-응답 모델은 2개의 입력을 가집니다. 하나는 자연어 질문이고, 또 하나는 답변에 필요한 정보가 담겨 있는 텍스트(예를 들어 뉴스 기사)입니다. 그러면 모델은 답을 출력해야 합니다. 가장 간단한 구조는 미리 정의한 어휘 사전에서 소프트맥스 함수를 통해 한 단어로 된 답을 출력하는 것입니다.

* 다음은 함수형 API를 사용하여 이런 모델을 만드는 예입니다. 텍스트와 질문을 벡터로 인코딩하여 독립된 입력 2개를 정의합니다. 그 다음 이 벡터를 연결하고 그 위에 소프트맥스 분류기를 추가합니다.

In [6]:
# 2개의 입력을 가진 질문-응답 모델의 함수형 API구현하기
# implement functional API of question-answer model with two inputs
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

# 텍스트 입력은 길이가 정해지지 않은 시퀀스입니다. 입력 이름을 지정할 수 있습니다.
text_input = Input(shape=(None,), dtype='int32', name='text')

embedded_text = layers.Embedding(
    # 입력을 크기가 64인 벡터의 시퀀스로 임베딩합니다.
    text_vocabulary_size, 64)(text_input)

# LSTM을 사용하여 이 벡터들을 하나의 벡터로 인코딩합니다.
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,),
                       dtype='int32',
                       name='question') # 질문도 동일한 과정을 거칩니다(층 객체는 다릅니다).

embedded_question = layers.Embedding(
    question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

# 인코딩된 질문과 텍스트를 연결합니다.
concatenated = layers.concatenate([encoded_text, encoded_question],
                     axis=-1)

# 소프트맥스 분류기를 추가합니다.
answer = layers.Dense(answer_vocabulary_size,
                            activation='softmax')(concatenated)
# 모델 객체를 만들고 2개의 입력과 출력을 주입합니다.
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

* 그럼 이렇게 입력이 2개인 모델은 어떻게 훈련할까요? 두 가지 방식이 있습니다. 넘파이 배열의 리스트를 주입하거나 입력 이름과 넘파이 배열로 이루어진 딕셔너리를 모델의 입력으로 주입할 수 있습니다. 당연하게 두 번째 방식은 입력 이름을 설정했을 때 사용할 수 있습니다.

In [7]:
# 다중 입력 모델에 데이터 주입하기
# Data injection in multi-input model
import numpy as np
from tensorflow.keras.utils import to_categorical

num_samples = 1000
max_length = 100

# 랜덤한 넘파이 데이터를 생성합니다.
text = np.random.randint(1, text_vocabulary_size,
                         size=(num_samples, max_length))

question = np.random.randint(1, question_vocabulary_size,
                             size=(num_samples, max_length))

answers = np.random.randint(0, answer_vocabulary_size, size=num_samples)

# 답은 정수가 아닌 원-핫 인코딩된 벡터입니다.
answers = to_categorical(answers)

# 리스트 입력을 사용하여 학습합니다.
model.fit([text, question], answers, epochs=10, batch_size=128)

# 딕셔너리 입력을 사용하여 학습합니다(입력 이름을 지정했을 때만 사용할 수 있습니다).
model.fit({'text': text, 'question': question}, answers,
           epochs=10, batch_size=128)

Epoch 1/10
8/8 [==============================] - 8s 168ms/step - loss: 6.2147 - acc: 0.0010
Epoch 2/10
8/8 [==============================] - 2s 200ms/step - loss: 6.1931 - acc: 0.0360
Epoch 3/10
8/8 [==============================] - 1s 149ms/step - loss: 6.1299 - acc: 0.0090
Epoch 4/10
8/8 [==============================] - 1s 155ms/step - loss: 6.0547 - acc: 0.0070
Epoch 5/10
8/8 [==============================] - 1s 181ms/step - loss: 6.0011 - acc: 0.0080
Epoch 6/10
8/8 [==============================] - 1s 143ms/step - loss: 5.9217 - acc: 0.0060
Epoch 7/10
8/8 [==============================] - 1s 148ms/step - loss: 5.8352 - acc: 0.0090
Epoch 8/10
8/8 [==============================] - 1s 174ms/step - loss: 5.7370 - acc: 0.0200
Epoch 9/10
8/8 [==============================] - 1s 151ms/step - loss: 5.6496 - acc: 0.0310
Epoch 10/10
8/8 [==============================] - 1s 143ms/step - loss: 5.5731 - acc: 0.0360
Epoch 1/10
8/8 [==============================] - 5s 186ms/step - los

### 7.1.3 다중 출력 모델
* 같은 식으로 함수형 API 모델을 사용하여 다중 출력(또는 다중 머리) 모델을 만들 수 있습니다. 간단한 예는 데이터에 있는 여러 속성을 동시에 예측하는 네트워크입니다. 예를 들어 소셜 미디어에서 익명 사용자의 포스트를 입력으로 받아 그 사람의 나이, 성별, 소득 수준 등을 예측합니다.

In [8]:
# 3개의 출력을 가진 함수형 API 구현하기
# implement functional API with having three outputs

from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model
vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(vocabulary_size, 256)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

# 출력 층에 이름을 지정합니다.
age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups,
                                 activation='softmax',
                                 name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(posts_input,
              [age_prediction, income_prediction, gender_prediction])

* 이런 모델을 훈련하려면 네트워크 출력마다 다른 손실 함수를 지정해야 합니다. 예를 들어 나이 예측은 스칼라 회귀 문제이지만 성별 예측은 이진 클래스 문제라 훈련 방식이 다릅니다. 경사 하강법은 하나의 스칼라 값을 최소화하기 때문에 모델을 훈련하려면 이 손실들을 하나의 값으로 합쳐야 합니다. 손실 값을 합치는 가장 간단한 방법은 모두 더하는 것입니다. 케라스에서는 compile 메서드에 리스트나 딕셔너리를 사용하여 출력마다 다른 손실을 지정할 수 있습니다. 계산된 손실 값은 전체 손실 하나로 더해지고 훈련 과정을 통해 최소화됩니다.

In [9]:
# 다중 출력 모델의 컴파일 옵션: 다중 손실
# compile option of multiple output models: multiple loss
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])
# 위와 동일합니다(출력 층에 이름을 지정했을 때만 사용할 수 있습니다).
model.compile(optimizer='rmsprop',
              loss={'age': 'mse',
                    'income': 'categorical_crosssentropy',
                    'gender': 'binary_crossentropy'})

* 손실 값이 많이 불균형하면 모델이 개별 손실이 가장 큰 작업에 치우쳐 표현을 최적화할 것입니다. 그 결과 다른 작업들은 손해를 입습니다. 이를 해결하기 위해 손실 값이 최종 손실에 기여하는 수준을 지정할 수 있습니다. 특히 손실 값의 스케일이 다를 때 유용합니다. 예를 들어 다음과 같이 가정해 보죠. 나이 회귀 작업에 사용되는 평균 제곱 오차(MSE: Mean Square Error) 손실은 일반적으로 3~5 사이의 값을 가집니다. 반면에 성별 분류 작업에 사용되는 크로스엔트로피 손실은 0.1 정도로 낮습니다. 이런 환경에서 손실에 균형을 맞추려면 크로스엔트로피 손실에 가중치 10을 주고 MSE 손실에 가중치 0.25를 줄 수 있습니다.

In [12]:
# 다중 출력 모델의 컴파일 옵션: 손실 가중치
# compile option of multiple output models: loss weight
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
              loss_weights=[0.25, 1., 10.])

# 위와 동일합니다(출력 층에 이름을 지정했을 때만 사용할 수 있습니다).
model.compile(optimizer='rmsprop',
              loss={'age': 'mse',
                    'income': 'categorical_crosssentropy',
                    'gender': 'binary_crossentropy'},
              loss_weights={'age': 0.25,
                            'income': 1.,
                            'gender': 10.})

* 다중 입력 모델과 마찬가지로 넘파이 배열의 리스트나 딕셔너리를 모델에 전달하여 훈련합니다.

In [ ]:
# 다중 출력 모델에 데이터 주입하기
# Inject data of multiple output models

# age_targets, income_targets, gender_targets가 넘파이 배열이라고 가정합니다.
model.fit(posts, [age_targets, income_targets, gender_targets],
          epochs=10, batch_size=64)
# 위와 동일합니다(출력 층에 이름을 지정했을 때만 사용할 수 있습니다).
model.fit(posts, {'age': age_targets,
                  'income': income_targets,
                  'gender': gender_targets},
          epochs=10, batch_size=64)

### 7.1.4 층으로 구성된 비순환 유향 그래프
* 함수형 API를 사용하면 다중 입력이나 다중 출력 모델뿐만 아니라 내부 토폴로지가 복잡한 네트워크도 만들 수 있습니다. 케라스의 신경망은 층으로 구성된 어떤 **비순환 유향 그래프(directed acyclic graph)** 도 만들 수 있습니다. 비순환이라는 것이 중요합니다. 다시 말해 이 그래프는 원형을 띨 수 없습니다. 텐서 x가 자기 자신을 출력하는 층의 입력이 될 수 없습니다. 만들 수 있는 루프(즉 순환 연결)는 순환 층의 내부에 있는 것뿐입니다.
* 그래프로 구현된 몇 개의 신경망 컴포넌트가 널리 사용됩니다. 가장 유명한 2개는 인셉션 모듈과 잔차 연결입니다. 케라스에서 이 2개의 컴포넌트를 어떻게 구현하는지 살펴보겠습니다. 함수형 API를 사용하여 층의 그래프를 만드는 방법을 이해하는 데 도움이 될 것입니다.

In [ ]:
# 인셉션 모듈(Inception module)
from tensorflow.keras import layers
# 모든 가지는 동일한 스트라이드(2)를 사용합니다.
# 출력 크기를 동일하게 만들어 하나로 합치기 위해서입니다.
branch_a = layers.Conv2D(128, 1,
                         activation='relu', strides=2)(x)
# 이 가지에서는 두 번째 합성곱 층에서 스트라이드를 적용합니다.
branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_b)
# 이 가지에서는 평균 폴링 층에서 스트라이드를 적용합니다.
branch_c = layers.AveragePooling2D(3, strides=2)(x)
branch_c = layers.Conv2D(128, 3, activation='relu')(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu')(x)
branch_d = layers.Conv2D(128, 3, activation='relu')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_d)
# 모든 가지의 출력을 연결하여 모듈의 출력을 만듭니다.
output = layers.concatenate(
    [branch_a, branch_b, branch_c, branch_d], axis=-1)

#### 잔차 연결(residual connection)

* 다음 코드는 케라스에서 특성 맵의 크기가 같을 때 원본을 그대로 사용하는 잔차 연결을 구현한 예입니다. 여기서는 입력 x가 4D 텐서라고 가정합니다.

In [ ]:
# 특성 맵의 크기가 같을 때 잔차 연결 구현하기
# implement residual connection when property maps are the same size

from tensorflow.keras import layers
x = ...
# x에 어떤 변환을 적용합니다.
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
# 원본 x를 출력 특성에 더합니다.
y = layers.add([y, x])

* 다음은 특성 맵의 크기가 다를 때 선형 변환을 사용하여 잔차 연결을 구현한 예입니다(여기에서도 입력 x가 4D 텐서라고 가정합니다).

In [ ]:
# 특성 맵의 크기가 다를 때 선형 변환을 사용하여 잔차 연결 구현하기
# implement residual connection by using linear transformations when property maps are the different size

from tensorflow.keras import layers
x = ...
# x에 어떤 변환을 적용합니다.
y = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same')(y)
y = layers.MaxPooling2D(2, strides=2)(y)

# y와 크기를 맞추기 위해 1 x 1 합성곱을 사용해여 원본 텐서 x를 다운샘플링합니다.
residual = layers.Conv2D(128, 1, strides=2, padding='same'(x))
# 다운샘플링된 x를 출력 특성에 더합니다.
y = layers.add([y, residual])

### 7.1.5 층 가중치 공유
* 함수형 API의 중요한 또 하나의 기능은 층 객체를 여러 번 재사용할 수 있다는 것입니다. 층 객체를 두 번 호출하면 새로운 층 객체를 만들지 않고 각 호출에 동일한 가중치를 재사용합니다. 이런 기능 때문에 공유 가지를 가진 모델을 만들 수 있습니다. 이런 가지는 같은 가중치를 공유하고 같은 연산을 수행합니다. 다시 말해 같은 표현을 공유하고 이런 표현을 다른 입력에서 함께 학습합니다.
* 예를 들어 두 문장 사이의 의미가 비슷한지 측정하는 모델을 가정해 보죠. 이 모델은 2개의 입력을 받고 0과 1 사이의 점수를 출력합니다. 0은 관련 없는 문장을 의미하고 1은 두 문장이 동일하거나 재구성되었다는 것을 의미합니다. 이런 모델은 대화 시스템(dialog system)에서 자연어 질의에 대한 중복 제거를 포함하여 많은 애플리케이션에서 유용하게 사용될 수 있습니다.
* 이런 문제에서는 두 입력 시퀀스가 바뀔 수 있습니다. 의미가 비슷하다는 것은 대칭적인 관계이기 때문입니다. A에서 B에 대한 유사도는 B에서 A에 대한 유사도와 같습니다. 이런 이유 때문에 각 입력 문장을 처리하는 2개의 독립된 모델을 학습하는 것은 이치에 맞지 않습니다. 그 대신 하나의 LSTM 층으로 양쪽을 모두 처리하는 것이 좋습니다. 이 LSTM 층의 표현(가중치)은 두 입력에 대해 함께 학습됩니다. 이를 샴 LSTM(Siamese LSTM)모델 또는 공유 LSTM이라고 부릅니다.
* 다음은 케라스의 함수형 API로 공유 층(재사용 층)을 사용하는 모델을 구현하는 예입니다.

In [ ]:
# 함수형 API로 공유 층(재사용 층)을 사용하는 모델 구현하기
# implement a model by using a shared layer(reusable layer) as a functional API
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model
# LSTM 층 객체 하나를 만듭니다.
lstm = layers.LSTM(32)
# 모델의 왼쪽 가지를 구성합니다. 입력은 크기가 128인 벡터의 가변 길이 시퀀스입니다.
left_input = Input(shape=(None, 128))
left_output = lstm(left_input)

# 모델의 오른쪽 가지를 구성합니다. 기존 층 객체를 호출하면 가중치가 재사용됩니다.
right_input = Input(shape=(None, 128))
right_output = lstm(right_input)

# 맨 위에 분류기를 놓습니다.
merged = layers.concatenate([left_output, right_output], axis=-1)
predictions = layers.Dense(1, activation='sigmoid')(merged)

# 모델 객체를 만들고 훈련합니다. 이런 모델을 훈련하면 LSTM 층의 가중치는 양쪽 입력을 바탕으로 업데이트합니다.
model = Model([left_input, right_input], predictions)
model.fit([left_data, right_data], targets)

* 당연하게 층 객체는 한 번 이상 사용할 수 있습니다. 같은 가중치를 재사용하면서 얼마든지 여러번 호출할 수 있습니다.

### 7.1.6 층과 모델
* 함수형 API에서는 모델을 층처럼 사용할 수 있습니다. 모델을 '커다란 층'으로 생각해도 됩니다. Sequential 클래스와 Model 클래스에서 모두 동일합니다. 이 말은 입력 텐서로 모델을 호출해서 출력 텐서를 얻을 수 있다는 뜻입니다.
```python
y = model(x)
```
* 모델에서 입력 텐서와 출력 텐서가 여러 개이면 텐서의 리스트로 호출합니다.
```python
y1, y2 = model([x1, x2])
```
* 모델 객체를 호출할 때 모델의 가중치가 재사용됩니다. 층 객체를 호출할 때와 정확히 같습니다. 층 객체나 모델 객체나 객체를 호출하는 것은 항상 그 객체가 가진 학습된 표현을 재사용합니다.
* 모델 객체를 재사용하는 간단한 실전 예는 듀얼 카메라에서 입력을 받는 비전 모델입니다. 두 카메라가 몇 센티미터(1인치) 간격을 두고 나란히 있습니다. 이런 모델은 깊이를 감지할 수 있습니다. 많은 애플리케이션에서 유용한 기능입니다. 왼쪽 카메라와 오른쪽 카메라에서 시각적 특징을 추출하여 합치기 위해 2개의 독립된 모델을 사용할 필요가 없습니다. 두 입력에 저수준 처리 과정이 공유될 수 있습니다. 다시 말해 가중치가 같고 동일한 표현을 공유하는 층을 사용합니다. 다음은 케라스에서 샴 비전 모델(공유 합성곱 기반 층)을 구현하는 예입니다.

In [2]:
# 샴 비전 모델(공유 합성곱 기반 층) 구현
# implement siamese vision model(shared convolution based layer)
from tensorflow.keras import layers
from tensorflow.keras import applications
from tensorflow.keras import Input

# 이미지 처리 기본 모델은 엑셉션 네트워크입니다(합성곱 기반 층만 사용합니다).
xception_base = applications.Xception(weights=None,
                                      include_top=False)
# 입력은 250 X 250 RGB 이미지입니다.
left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

# 같은 비전 모델을 두 번 호출합니다.
left_features = xception_base(left_input)
right_features = xception_base(right_input)

# 합쳐진 특성은 오른쪽 입력과 왼쪽 입력에서 얻은 정보를 담고 있습니다.
merged_features = layers.concatenate(
    [left_features, right_features], axis=-1)

### 7.1.7 정리
* 이것으로 케라스의 함수형 API 소개를 마칩니다. 이는 고급 심층 신경망 구조를 구축하기 위해 필수적인 도구입니다. 여기에서 다음 내용을 배웠습니다.
    * 차례대로 층을 쌓는 것 이상이 필요할 때는 Sequential API를 사용하지 않습니다.
    * 함수형 API를 사용하여 다중 입력, 다중 출력, 복잡한 네트워크 토폴로지를 갖는 케라스 모델을 만드는 방법
    * 다른 네트워크 가지에서 같은 층이나 모델 객체를 여러 번 호출하여 가중치를 재사용하는 방법

## 7.2 케라스 콜백과 텐서보드를 사용한 딥러닝 모델 검사와 모니터링
* 이 절에서는 훈련하는 동안 모델 내부에서 일어나는 일을 조사하고 제어하는 방법을 살펴보겠습니다. 대규모 데이터셋에서 model.fit()이나 model.fit_generator()로 수십 번의 에포크(epoke)를 실행하는 것은 종이 비행기를 날리는 것과 조금 비슷합니다. 일단 손을 떠나면 종이 비행기 경로와 착륙 지점을 제어할 방법이 없습니다. 나쁜 결과를 피하려면 (그래서 종이 비행기를 낭비하지 않으려면)  종이 비행기 대신 다른 것을 사용하는 것이 좋습니다. 드론은 주변 환경을 감지한 데이터를 조작부에 전달하여 현재 상태를 바탕으로 자동으로 운전합니다. 앞으로 소개할 기법은model.fit() 호출을 종이 비행기에서 스스로 판단하고 동적으로 결정하는 똑똑한 자동 드론으로 바꾸어 줄 것입니다.

### 7.2.1 콜백을 사용하여 모델의 훈련 과정 제어하기

* 모델을 훈련할 때 미리 예상할 수 없는 것들이 많습니다. 특히 최적의 검증 손실을 얻기 위해 얼마나 많은 에포크가 필요한지 알지 못합니다. 지금까지 예제는 적절한 훈련 에포크를 알아내기 위해 첫 번째 실행에서 과대적합이 시작될 때까지 충분한 에포크로 훈련했습니다. 그런 다음 최적의 에포크 횟수로 처음부터 새로운 훈련을 시작했습니다.당연히 이런 방식은 낭비가 많습니다.

* 다음은 콜백을 사용하는 몇 가지 사례입니다.
    * **모델 체크포인트 저장**: 훈련하는 동안 어떤 지점에서 모델의 현재 가중치를 저장합니다.
    * **조기 종료**(early stopping): 검증 손실이 더 이상 향상되지 않을 때 훈련을 중지합니다(물론 훈련하는 동안 얻은 가장 좋은 모델을 저장합니다).
    * **훈련하는 동안 하이퍼파라미터 값을 동적으로 조정합니다**: 옵티마이저(optimizer)의 학습률 같은 경우입니다.
    * **훈련과 검증 지표를 로그에 기록하거나 모델이 학습한 표현이 업데이터될 때마다 시각화합니다**: 앞서 보았던 케라스의 진행 표시줄(progress bar)이 하나의 콜백입니다!
    
* keras.callbacks 모듈은 많은 내장 콜백을 포함하고 있습니다(다음은 전체 리스트가 아닙니다).
    keras.callbacks.ModelCheckpoint
    keras.callbacks.EarlyStopping
    keras.callbacks.LearningRateScheduler
    keras.callbacks.ReduceLROnPlateau
    keras.callbacks.CSVLogger
* 콜백 사용법을 익히기 위해 ModelCheckpoint, EarlyStopping, ReduceLROnPlateau를 사용한 예를 살펴봅시다.

#### ModelCheckpoint와 EarlyStopping 콜백
* EarlyStopping 콜백을 사용하면 정해진 에포크 동안 모니터링 지표가 향상되지 않을 때 훈련을 중지할 수 있습니다. 예를 들어 과대적합이 시작되자마자 훈련을 중지할 수 있습니다. 따라서 에포크 횟수를 줄여 다시 모델을 훈련할 필요가 없습니다. 일반적으로 이 콜백은 훈련하는 동안 모델을 계속 지정해 주는 ModelCheckpoint와 함께 사용합니다. (선택적으로 지금까지 가정 좋은 모델만 저장할 수 있습니다. 에포크 마지막에 다다랐을 때 최고 성능을 달성한 모델입니다.)

In [ ]:
# ModelCheckpoint와 EarlyStopping 콜백 구현하기
# implement ModelCheckpoint and EarlyStopping callback
import keras
# fit() 메서드의 callbacks 매개변수를 사용하여 콜백의 리스트를 
# 모델로 전달합니다. 몇 개의 콜백이라도 전달할 수 있습니다.
callback_list = [  
    # 성능 향상이 멈추면 훈련을 중지합니다.
    keras.callbacks.EarlyStopping(
        # 모델의 검증 정확도를 모니터링합니다.
        monitor='val_acc'
        # 1 에포크보다 더 길게 (즉 2 에포크 동안) 정확도가 향상되지 않으면 훈련이 중지됩니다.
        patience=1,
    ),
    # 에포크마다 현재 가중치를 저장합니다.
    keras.callbacks.ModelCheckpoint(
        # 모델 파일의 경로
        filepath='my_model.h5',
        # 이 두 매개변수는 val_loss가 좋아지지 않으면 모델 파일을 덮어쓰지
        # 않는다는 뜻입니다. 훈련하는 동안 가장 좋은 모델이 저장됩니다.
        monitor='val_loss',
        save_best_only=True,
    )
]

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc']) # 정확도를 모니터링하므로 모델 지표에 포함되어야 합니다.
# 콜백이 검증 손실과 검증 정확도를 모니터링하기 때문에
# validation_data 매개변수에 검증 데이터를 전달해야 합니다.
model.fit(x, y,
          epochs=10,
          batch_size=32,
          callbacks=callbacks_list,
          validation_data=(x_val, y_val))

#### ReduceLROnPlateau 콜백
* 이 콜백을 사용하면 검증 손실이 향상되지 않을 때 학습률을 작게 할 수 있습니다. 손실 곡선이 평탄할 때 학습률을 작게 하거나 크게 하면 훈련 도중 지역 최솟값에서 효과적으로 빠져나올 수 있습니다. 다음은 ReduceLROnPlateau 콜백을 사용하는 예입니다.

In [ ]:
# ReduceLROnPlateau 콜백 구현하기
# implement ReduceLROnPlateau callback
callbacks_list = [
    keras.callbacks.ReduceLROnPlateau(
        # 모델의 검증 손실을 모니터링합니다.
        monitor='val_loss',
        # 콜백이 호출될 때 학습률을 10배로 줄입니다.
        factor=0.1,
        # 검증 손실이 10 에포크 동안 좋아지지 않으면 콜백이 호출됩니다.
        patience=10,
    )
]

# 콜백이 검증 손실을 모니터링하기 때문에 validation_data
# 매개변수에 검증 데이터를 전달해야 합니다.
model.fit(x, y,
          epochs=10,
          batch_size=32,
          callbacks=callbacks_list,
          validation_data=(x_val, y_val))

#### 자신만의 콜백 만들기
* 내장 콜백에서 제공하지 않는 특수한 행동이 훈련 도중 필요하면 자신만의 콜백을 만들 수 있습니다. 콜백은 keras.callbacks.Callback 클래스를 상속받아 구현합니다. 그다음 훈련하는 동안 호출될 여러 지점을 나타내기 위해 약속된 다음 메서드를 구현합니다.
```python
# 각 에포크가 시작할 때 호출합니다.
on_epoch_begin
# 각 에포크가 끝날 때 호출합니다.
on_epoch_end
# 각 배치 처리가 시작되기 전에 호출합니다.
on_batch_begin
# 각 배치 처리가 끝난 후에 호출합니다.
on_batch_end
# 훈련이 시작될 때 호출합니다.
on_train_begin
# 훈련이 끝날 때 호출합니다.
on_train_end
```
* 이 메서드들은 모두 logs 매개변수와 함께 호출됩니다. 이 매개변수에는 이전 배치, 에포크에 대한 훈련과 검증 측정값이 담겨 있는 딕셔너리가 전달됩니다. 또 콜백은 다음 속성을 참조할 수 있습니다.
    * self.model: 콜백을 호출하는 모델 객체
    * self.validation_data: fit() 메서드에 전달된 검증 데이터
* 다음은 매 에포크의 끝에서 검증 세트의 첫 번째 샘플로 모델에 있는 모든 층의 활성화 출력을 계산하여 (넘파이 배열로) 디스크에 저장하는 자작 콜백의 예입니다.

In [ ]:
# 넘파이 배열 사용해 자신만의 콜백 만들기
# create your own callback by using numpy array
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):
    # 호출하는 모델에 대한 정보를 전달하기 위해 훈련하기 전에 호출됩니다.
    def set_model(self, model):
        self.model = model
        layer_outputs = [layer.output for layer in model.layers]
        self.activations_model = keras.models.Model(model.input,
                                                    layer_outputs) # 각 층의 활성화 출력을 반환하는 Model의 객체입니다.
        
    def on_epoch_end(self, epoch, logs=None):
        if self.validation_data is None:
            raise RuntimeError('Requires validation_data.')
            
            # 검증 데이터의 첫 번째 샘플을 가져옵니다.
            validation_sample = self.validation_data[0][0:1]
            activations = self.activations_model.predict(validation_sample)
            # 배열을 디스크에 저장합니다.
            f = open('activations_at_epoch_' + str(epoch) + '.npz', 'wb')
            np.savez(f, activations)
            f.close()

* 이것이 콜백에 대해 알아야 할 전부입니다. 기술적인 세부 사항은 쉽게 찾아볼 수 있습니다. 이제 훈련하는 동안 케라스 모델에 어떤 종류의 로깅이나 프로그래밍된 간섭을 수행할 수 있을 것입니다.

### 7.2.2 텐서보드 소개: 텐서플로의 시각화 프레임워크
* 좋은 연구를 하거나 좋은 모델을 개발하려면 실험하는 모델 내부에서 어떤 일이 일어나는지 자주 그리고 많은 피드백을 받아야 합니다. 그것이 실험을 하는 목적입니다. 모델이 얼마나 잘 작동하는지 가능한 많은 정보를 얻는 것입니다. 발전은 반복되는 프로세스 또는 루프를 통해 일어납니다. 한 아이디어가 떠오르면 이 아이디어를 검증할 실험을 계획합니다. 실험을 수행하고 생성된 정보를 가공합니다. 이 정보는 다음 아이디어에 영감을 줍니다. 이 루프를 더 많이 실행할수록 아이디어는 더 정제되고 강력해질 것입니다. 케라스는 가능한 최단 시간에 아이디어를 실험으로 구현하도록 도와줍니다. 고속의 GPU가 가능한 빠르게 실험의 결과를 얻도록 도와줄 것입니다. 그렇다면 실험 결과는 어떻게 처리할까요? 바로 텐서보드가 하는 일입니다.

* 이 절에서는 텐서플로와 함께 제공되는 브라우저 기반 시각화 도구인 텐서보드를 소개합니다. 텐서플로 백엔드로 케라스를 설정할 경우에만 케라스 모델에서 사용할 수 있습니다.
* 텐서보드의 핵심 목적은 훈련 모델의 내부에서 일어나는 모든 것을 시각적으로 모니터링할 수 있도록 돕는 것입니다. 모델의 최종 손실 외에 더 많은 정보를 모니터링하면 모델 작동에 대한 명확한 그림을 그릴 수 있습니다. 결국 모델을 더 빠르게 개선할 수 있습니다. 텐서보드는 여러 가지 멋진 기능을 제공합니다. 모두 브라우저에서 작동합니다.
    * 훈련하는 동안 측정 지표를 시각적으로 모니터링합니다.
    * 모델 구조를 시각화합니다.
    * 활성화 출력과 그래디언트의 히스토그램을 그립니다.
    * 3D로 임베딩을 표현합니다.
* 간단한 예를 사용하여 이 기능들을 실습해 보죠. IMDB 감성 분석 문제를 위해 1D 컨브넷을 훈련하겠습니다.
* 이 모델은 6장 마지막 절에서 보았던 것과 비슷합니다. IMDB 어휘 사전에서 빈도가 높은 2,000개 단어만 사용하겠습니다. 이렇게 하면 단어 임베딩(word embedding)을 시각화하기가 조금 더 쉽습니다.

In [2]:
# 텐서보드를 사용한 텍스트 분류 모델
# Text classification model by using tensorboard
import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

# 특성으로 사용할 단어의 수
max_features = 2000
# 사용할 텍스트의 길이(가장 빈번한 max_features개의 단어만 사용합니다.)
max_len = 500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=max_len)
x_test = sequence.pad_sequences(x_test, maxlen=max_len)

model = tensorflow.keras.models.Sequential()
model.add(layers.Embedding(max_features, 128,
                           input_length=max_len,
                           name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
c:\users\wnstj\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
c:\users\wnstj\appdata\local\programs\python\python39\lib\site-packages\tensorflow\python\keras\datasets\imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays wit

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embed (Embedding)            (None, 500, 128)          256000    
_________________________________________________________________
conv1d (Conv1D)              (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 98, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 291,937
Trainable params: 291,937
Non-trainable params: 0
__________________________________________________

* 텐서보드를 시작하기 전에 로그 파일이 저장될 디렉터리를 만들어야 합니다.
```bash
$ mkdir my_log_dir
```

* TensorBoard 콜백 객체와 함께 훈련을 시작해 보죠. 이 콜백은 지정된 디스크 위치에 로그 이벤트를 기록할 것입니다.

In [10]:
# 텐서보드 콜백과 함께 모델 훈련하기
# train a model with TensorBoard callback

callbacks = [
    tensorflow.keras.callbacks.TensorBoard(
        # 로그 파일이 기록될 위치입니다.
        log_dir='my_log_dir',
        # 1 에포크마다 활성화 출력의 히스토그램을 기록합니다.
        histogram_freq=1,
        # 1 에포크마다 임베딩 데이터를 기록합니다.
        embeddings_freq=1,
    )
]

history = model.fit(x_train, y_train,
                    epochs=20,
                    batch_size=128,
                    validation_split=0.2,
                    callbacks=callbacks)

Epoch 1/20
157/157 [==============================] - 98s 613ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 2/20
157/157 [==============================] - 91s 581ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 3/20
157/157 [==============================] - 89s 569ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 4/20
157/157 [==============================] - 88s 562ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 5/20
157/157 [==============================] - 91s 579ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 6/20
157/157 [==============================] - 86s 547ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 7/20
157/157 [==============================] - 91s 582ms/step - loss: 7.7364 - acc: 0.4985 - val_loss: 7.6168 - val_acc: 0.5062
Epoch 8/20
157/157 [==============================] - 8

* 이제 명령행에서 콜백이 사용되는 로그 디렉터리를 지정하여 텐서보드 서버를 실행합니다. 텐서플로를 설치했다면 (예를 들어 pip 명령으로 설치했다면) 텐서보드 유틸리티는 자동으로 설치되었을 것입니다.
```bash
$ tensorboard --logdir=my_log_dir
```
* 그 다음 브라우저에서 `http://localhost:6006` 주소에 접속하면 모델의 훈련 결과를 확인할 수 있습니다. 훈련 지표와 검증 지표를 실시간 그래프로 볼 수 있고 Histograms 탭에서 층의 활성화 출력을 멋진 히스토그램 그래프로 볼 수 있습니다.

* PROJECTOR 탭에서 입력 어휘 사전에 있는 단어  2,000개의 임베딩 위치와 공간상 관계를 조사할 수 있습니다. 이 임베딩은 첫 번째 Embedding 층이 학습한 것입니다. 임베딩 공간이 128차원이기 때문에 텐서보드는 우리가 선택한 차원 축소 알고리즘을 사용하여 자동으로 2D 또는 3D로 축소합니다. 주성분 분석(Principal Component Analysis, PCA) 또는 t-SNE(t-distributed Stochastic Neighbor Embedding)입니다. 점 구름(point cloud) 그래프에서 2개의 클러스터가 뚜렷하게 구분됩니다. 긍정적인 의미의 단어와 부정적인 의미의 단어입니다. 이 그래프는 주어진 문제에 완전히 특화된 모델의 목적에 맞게 임베딩이 훈련되었다는 것을 확실하게 보여 줍니다. 이것이 사전 훈련된 보편적인 단어 임베딩이 드물게 사용되는 이유입니다.

* Graphs 탭은 케라스 모델을 구성하는 저수준 텐서플로 연산의 그래프를 시각화합니다. 여기서 볼 수 있듯이 예상보다 훨씬 복잡합니다. 케라스에서 만든 모델은 간단해 보이지만 (기본 층을 몇 개 쌓았습니다) 무대 뒤에서는 상당히 복잡한 그래프 구조가 만들어집니다. 그래프의 많은 부분은 경사 하강법과 관련이 있습니다. 보이는 것과 만들어야 하는 것 사이의 이런 복잡도 차이 때문에 모든 것을 밑바닥부터 텐서플로를 사용하여 만드는 대신 케라스를 사용하여 모델을 만듭니다. 케라스는 작업 흐름을 극적으로 간단하게 만들어 줍니다.

In [3]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='model.png')

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


* 층 그래프에 크기 정보를 추가할 수 있습니다. 다음은 plot_model 함수와 show_shapes 매개변수를 사용하여 모델의 그래프를 그립니다.

In [4]:
from tensorflow.keras.utils import plot_model
plot_model(model, show_shapes=True, to_file='model.png')

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


### 7.2.3 정리
* 케라스 콜백은 훈련하는 동안 모델을 모니터링하고 모델 상태를 바탕으로 자동으로 작업을 수행하는 손쉬운 방법입니다.
* 텐서플로를 사용하면 텐서보드를 이용하여 모델 상황을 브라우저에서 시각화할 수 있습니다. 케라스 모델에서는 TensorBoard 콜백을 통해 사용합니다.

## 7.3 모델의 성능을 최대로 끌어올리기
* 단지 작동만 하는 모델이 필요하다면 그냥 이런저런 구조를 시도해 보아도 충분합니다. 이 절에서는 작동하는 수준을 넘어 아주 잘 작동하고 머신 러닝 경연 대회에서 우승하는 모델을 만들기 위해 꼭 알아야 할 기법을 소개하겠습니다. 이런 기법을 사용하면 최고의 딥러닝 모델을 만들 수 있을 것입니다.

### 7.3.1 고급 구조 패턴
* 이전 절에서 중요한 디자인 패턴 하나를 자세히 소개했습니다. 잔차 연결입니다. 이외에도 꼭 알아야 할 디자인 패턴이 2개 더 있습니다. 정규화와 깊이별 분리 합성곱입니다. 이 패턴은 특히 고성능 심층 컨브넷을 만들 때 유용합니다. 하지만 보통 다른 종류의 구조에서도 많이 등장합니다.

#### 배치 정규화(Batch Normalization)
* **정규화**(normalization)는 머신 러닝 모델에 주입되는 샘플들을 균일하게 만드는 광범위한 방법입니다. 이 방법은 모델이 학습하고 새로운 데이터에 잘 일반화되도록 돕습니다. 데이터 정규화의 가장 일반적인 형태는 이미 이 책에서 여러 번 나왔습니다. 데이터에서 평균을 빼서 데이터를 원점에 맞추고 표준 편차로 나누어 데이터의 분산을 1로 만듭니다. 데이터가 정규 분포(가우시안 분포)를 따른다 가정하고 이 분포를 원점에 맞추고 분산이 1이 되도록 조정한 것입니다.
```python
normalized_data = (data - np.mean(data, axis=...)) / np.std(data, axis=...)
```
* 이전 예제는 모델에 데이터를 주입하기 전에 정규화했습니다. 하지만 데이터 정규화는 네트워크에서 일어나는 모든 변환 후에도 고려되어야 합니다. Dense나 Conv2D 층에 들어가는 데이터의 평균이 0이고 분산이 1이더라도 출력되는 데이터가 동일한 분포를 가질 것이라고 기대하기 어렵습니다.

#### 깊이별 분리 합성곱(depthwise separable convolution)

* Conv2D를 대체하면서 더 가볍고(훈련할 모델 파라미터가 더 적고) 더 빨라(부동 소수 연산이 더 적고) 모델의 성능을 몇 퍼센트 포인트 높일 수 있는 층이 있다면 어떨까요? 이것이 **깊이별 분리 합성곱**(depthwise separable convolution) 층이 하는 일입니다(SeparableConv2D). 이 층은 입력 채널별로 따로따로 공간 방향의 합성곱을 수행합니다. 그다음 점별 합성곱(1 X 1 합성곱)을 통해 출력 채널을 합칩니다. 이는 공간 특성의 학습과 채널 방향 특성의 학습을 분리하는 효과를 냅니다. 입력에서 공간상 위치는 상관관계가 크지만 채널별로는 매우 독립적이라고 가정한다면 타당합니다. 이 방법은 모델 파라미터와 연산의 수를 크게 줄여 주기 때문에 작고 더 빠른 모델을 만듭니다. 합성곱을 통해 더 효율적으로 표현을 학습하기 때문에 적은 데이터로도 더 좋은 표현을 학습하고, 결국 성능이 더 높은 모델을 만듭니다.

* 이 장점은 제한된 데이터로 작은 모델을 처음부터 훈련시킬 때 특히 더 중요합니다. 다음은 작은 데이터셋에서 이미지 분류 문제(소프트맥스 분류)를 위한 가벼운 깊이별 분리 컨브넷을 만드는 예입니다.

In [ ]:
# 깊이별 분리 합성곱 구현하기
# implement depthwise separable convolution
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers

height = 64
width = 64
channels = 3
num_classes = 10

model = Sequential()
model.add(layers. SeparableConv2D(32, 3,
                                  activation='relu',
                                  input_shape=(height, width, channels,)))
model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation='relu'))
model.add(layers.SeparableConv2D(128, 3, activation='relu'))
model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dense(32, activaiton='relu'))

model.add(layers.Dense(num_classes, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

### 7.3.2 하이퍼파라미터 최적화(Hyperparameter Optimization)
* 딥러닝 모델을 만들 때 무작위로 보이는 결정을 많이 해야 합니다. 얼마나 많은 층을 쌓아야 할까요? 층마다 얼마나 많은 유닛이나 필터를 두어야 할까요? relu 활성화 함수를 사용해야 할까요? 아니면 다른 함수를 사용해야 할까요? 어떤 층 뒤에 BatchNormalization을 사용해야 할까요? 드롭아웃은 얼마나 해야 할까요? 등입니다. 이런 구조에 관련된 파라미터를 역전파로 훈련되는 모델 파라미터와 구분하여 **하이퍼파라미터**(hyperparameter)라고 부릅니다.
* 실제로 경험 많은 머신 러닝 엔지니어와 연구자는 하이퍼파라미터에 따라 작동하는 것과 작동하지 않는 것에 대한 직관을 가지고 있습니다. 하이퍼파라미터 튜닝에 관한 기술을 가지고 있는 셈입니다. 하지만 공식적인 규칙은 없습니다. 주어진 문제에서 최대의 성능을 얻고 싶다면 사람이 임으로 선택한 결정에 만족해서는 안 됩니다. 직감이 좋다고 하더라도 첫 번째 선택은 거의 항상 최적치가 아닙니다. 옵션을 수정하고 모델을 반복적으로 다시 훈련하여 선택 사항을 개선해야 합니다. 이것이 머신 러닝 엔지니어와 연구자들이 대부분의 시간을 쓰는 일입니다. 하지만 하루 종일 하이퍼파라미터를 수정하는 것은 사람이 할 일은 아닙니다. 기계에 위임하는 것이 더 낫습니다.
* 가능한 결정 공간을 자동적, 조직적, 규칙적 방법으로 탐색해야 합니다. 가능성 있는 구조를 탐색해서 실제 가장 높은 성능을 내는 것을 찾아야 합니다. 하이퍼파리미터 자동 최적화가 이에 관련된 분야입니다. 이는 하나의 연구 분야이며 중요한 분야입니다.
* 전형적인 하이퍼파라미터 최적화 과정은 다음과 같습니다.
    1. 일련의 하이퍼파리미터를 (자동으로) 선택합니다.
    2. 선택된 하이퍼파라미터로 모델을 만듭니다.
    3. 훈련 데이터에 학습하고 검증 데이터에서 최종 성능을 측정합니다.
    4. 다음으로 시도할 하이퍼파라미터를 (자동으로) 선택합니다.
    5. 이 과정을 반복합니다.
    6. 마지막으로 테스트 데이터에서 성능을 측정합니다.
* 주어진 하이퍼파라미터에서 얻은 검증 성능을 사용하여 다음 번에 시도할 하이퍼파라미터를 선택하는 알고리즘이 이 과정의 핵심입니다. 여러 가지 기법을 사용할 수 있습니다. 베이지안 최적화(bayesian optimization), 유전 알고리즘(genetic algorithms), 간단한 랜덤 탐색(random search) 등입니다.
* 모델의 가중치를 훈련하는 것은 비교적 쉽습니다. 미니 배치 데이터에 대한 손실 함수 값을 계산하고 역전파 알고리즘을 사용하여 올바른 방향으로 가중치를 이동하면 됩니다. 반면에 하이퍼파라미터를 업데이트하는 것은 매우 어려운 일입니다. 다음을 생각해 보죠.
    * 피드백 신호를 계산하는 것은 매우 비용이 많이 듭니다(이 하이퍼파라미터가 성능이 높은 모델을 만들어 낼까요?). 새로운 모델을 만들고 데이터셋을 사용하여 처음부터 다시 훈련해야 합니다.
    * 하이퍼파라미터 공간은 일반적으로 분리되어 있는 결정들로 채워집니다. 즉 연속적이지 않고 미분 가능하지 않습니다. 그러므로 하이퍼파라미터 공간에 경사 하강법을 사용할 수 없습니다. 그 대신 경사 하강법보다 훨씬 비효율적인 그래디언트-프리(gradient-free) 최적화 기법을 사용해야 합니다.
* 전체적으로 보았을 때 하이퍼파라미터 최적화는 어는 작업에서 최고의 모델을 얻거나 머신 러닝 경연 대회에서 우승하기 위한 강력한 도구입니다. 다음을 생각해 보죠. 오래 전에는 사람들이 얕은 머신 러닝 모델에 넣을 특성을 직접 만들었습니다. 이는 매우 최적화되지 않은 방법입니다. 요즘에는 딥러닝이 계층적인 특성 엔지니어링 작업을 자동화합니다. 수작업이 아니라 피드백 신호를 사용하여 특성이 학습됩니다. 어찌 보면 당연한 일입니다. 같은 식으로 수작업으로 모델 구조를 만드는 것이 아니라 이론에 근거하여 모델 구조를 최적화해야 합니다. 이 글을 쓰는 시점에는 하이퍼파라미터 자동 최적화 분야는 매우 초기 단계이고 미성숙합니다. 딥러닝도 수년 전에는 그랬습니다. 하지만 다음 몇 년 동안 크게 성장할 것으로 기대합니다.

### 7.3.3 모델 앙상블(Model Ensemble)
* **모델 앙상블**(model ensemble)은 가장 좋은 결과를 얻을 수 있는 또 다른 강력한 기법입니다. 앙상블은 여러 개 다른 모델의 예측을 합쳐서 더 좋은 예측을 만듭니다. 캐글 같은 머신 러닝 경연 대회에서는 우승자들이 대규모 모델 앙상블을 사용합니다. 이런 앙상블은 아주 뛰어난 단일 모델보다도 성능이 좋습니다.
* 앙상블은 독립적으로 훈련된 다른 종류의 좋은 모델이 각기 다른 장점을 가지고 있다는 가정을 바탕으로 합니다. 각 모델은 예측을 만들기 위해 조금씩 다른 측면을 바라봅니다. 데이터의 모든 면이 아니고 부분 특징입니다. 아마 장님과 코끼리에 관한 오래된 우화를 들어 보았을 텐데요.여러 명의 장님이 처음 코끼리를 만나면 코끼리를 더듬어 보고 이해하게 됩니다. 서로 코끼리의 다른 부분을 만집니다. 코나 다리처럼 딱 한 부분을 만집니다. 그리고 코끼리가 뱀 같다거나 기둥이나 나무 같다고 설명합니다.
* 이 장님들이 훈련 데이터의 매니폴드를 이해하려는 머신 러닝 모델입니다. 각자의 가정(고요한 모델 구조와 랜덤 가중치 초기화)을 이용하고 각자의 관점으로 이해합니다. 각 모델은 데이터의 일부분에 맞는 정답을 찾지만 완전한 정답은 아닙니다. 이들의 관점을 모으면 데이터를 훨씬 더 정확하게 묘사할 수 있습니다. ㅋ
* 분류 예를 들어 보죠. 분류기 예측을 (앙상블하기 위해) 합치는 가장 쉬운 방법은 추론할 때 나온 예측을 평균 내는 것입니다.

In [ ]:
# 앙상블하기 위해 분류기 예측 평균내기
# Average classifier predictions for ensemble

# 4개의 다른 모델을 사용하여 초기 예측을 계산합니다.
preds_a = model_a.predict(x_val)
preds_b = model_b.predict(x_val)
preds_c = model_c.predict(x_val)
preds_d = model_d.predict(x_val)
# 새로운 예측은 어떤 초기 예측보다 더 정확해야 합니다.
final_preds = 0.25 * (preds_a + preds_b + preds_c + preds_d)

* 이 방식은 분류기들이 어느 정도 비슷하게 좋을 때 잘 작동합니다. 분류기 중 하나가 다른 모델보다 월등히 나쁘면 최종 예측은 앙상블에 있는 가장 좋은 분류기만큼 좋지 않을 수 있습니다.
* 분류기를 앙상블하는 좋은 방법은 검증 데이터에서 학습된 가중치를 사용하여 가중 평균하는 것입니다. 전형적으로 분류기가 좋을수록 높은 가중치를 가지고 나쁜 분류기일수록 낮은 가중치를 갖습니다. 좋은 앙상블 가중치를 찾기 위해 랜덤 서치나 넬더-미드(Nelder-Mead) 방법 같은 간단한 최적화 알고리즘을 사용할 수 있습니다. 

In [ ]:
# 4개의 다른 모델을 사용하여 초기 예측을 계산합니다.
preds_a = model_a.predict(x_val)
preds_b = model_b.predict(x_val)
preds_c = model_c.predict(x_val)
preds_d = model_d.predict(x_val)
# 가중치(0.5, 0.25, 0.1, 0.15)는 경험적으로 학습되었다고 가정합니다.
final_preds = 0.5 * preds_a + 0.25 * preds_b + 0.1 * preds_c + 0.15 * preds_d)

* 이 외에도 여러 가지 변종이 있습니다. 예를 들어 예측의 지수 값을 평균할 수 있습니다. 일반적으로 검증 데이터에서 찾은 최적의 가중치로 단순하게 가중 평균하는 방법이 좋은 기본값입니다.